# Python Generators & Generator Expressions — from zero to hero 🧵⚙️

Welcome! This notebook teaches **generators** and **generator expressions** with
clear explanations, diagrams-in-your-head™, and lots of bite‑sized exercises.

> Tip: Solutions are hidden under clickable “**Show solution**” blocks.

## Table of Contents
1. What is a generator?
2. `yield` — the pause button
3. Iteration protocol: `iter()` & `next()`
4. Generators vs. lists (speed & memory)
5. Generator expressions `(x for x in ...)`
6. Chaining generators (pipelines)
7. Useful patterns & `itertools`
8. Common pitfalls
9. Exercises (easy → hard) + solutions



## 1. What is a generator?

A **generator** is a function that returns an *iterator*—it produces values **lazily**,
one at a time, using the `yield` keyword instead of `return`.

**Why care?**
- Saves memory: values are produced on demand (no big lists in RAM).
- Can model *streams* (files, sockets, infinite sequences).
- Composable: chain multiple simple steps into a data pipeline.


In [ ]:

def countdown(n: int):
    print("Start!")
    while n > 0:
        print("About to yield", n)
        yield n
        n -= 1
    print("Done.")

gen = countdown(3)
gen


In [ ]:

next(gen)


In [ ]:

list(gen)  # continue consuming remaining values



## 2. `yield` — the pause button

`yield` *pauses* the function and sends a value to the caller. On the next
iteration, execution resumes **right after** the `yield`.

Illustration:

```text
enter function → ... → yield value ──► caller gets value
                               ▲
                      next() resumes here
```


In [ ]:

def squares(n):
    for i in range(n):
        yield i, i*i

for i, sq in squares(5):
    print(i, sq)



## 3. The iteration protocol: `iter()` & `next()`

In Python, an **iterator** is any object with:
- `__iter__()` returning itself, and
- `__next__()` returning the next value or raising `StopIteration`.

Generators created by `yield` *already implement* this protocol.


In [ ]:

g = (x*x for x in range(3))  # a generator *expression*
it = iter(g)
print(next(it))
print(next(it))
print(next(it))
try:
    print(next(it))
except StopIteration:
    print("No more values.")



## 4. Generators vs. lists — memory usage

Lists store **all** items at once; generators compute items **on demand**.
For big ranges/streams, generators can be dramatically lighter on memory.


In [ ]:

import sys

lst = [i*i for i in range(100_000)]
gen = (i*i for i in range(100_000))

print("List size (bytes):", sys.getsizeof(lst))
print("Gen  size (bytes):", sys.getsizeof(gen))



## 5. Generator expressions `(expr for item in iterable if cond)`

They look like list comprehensions but with parentheses — producing values lazily.

**Examples:**

In [ ]:

nums = [1, 2, 3, 4, 5, 6]
evens_sq = (n*n for n in nums if n % 2 == 0)
print(evens_sq)       # just the generator object
print(list(evens_sq)) # consuming it



## 6. Chaining generators (pipelines)

You can build readable pipelines by composing small generator steps.


In [ ]:

def read_lines(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            yield line.rstrip("\n")

def grep(keyword, lines):
    for line in lines:
        if keyword.lower() in line.lower():
            yield line

def take(n, it):
    for i, x in enumerate(it):
        if i == n: 
            return
        yield x

# Demo with an in-memory "file"
data = ["Error: disk full", "ok", "warning", "error: fan", "OK"]
lines = (x for x in data)
pipeline = take(2, grep("error", lines))
list(pipeline)



## 7. Useful patterns & `itertools`

The `itertools` module provides fast building blocks that play perfectly with generators:
- `count(start=0, step=1)` — infinite counter
- `islice(iterable, stop)` — take first *n* items
- `chain(a, b, ...)` — chain multiple iterables
- `accumulate(iterable)` — running totals
- `groupby(iterable, key=...)` — group consecutive items


In [ ]:

from itertools import count, islice, chain, accumulate

first10 = list(islice(count(10, 2), 10))
chained = list(chain([1,2], [3,4], range(5,7)))
running = list(accumulate([1,2,3,4]))

first10, chained, running



## 8. Common pitfalls

1. **Single-use**: Once consumed, a generator is *exhausted*.
2. **Order of consumption** matters: if you partially consume, the next user starts later.
3. **Side effects** in generators can be surprising if iterated multiple times.
4. **Debugging**: Wrap with `list(...)` for a quick peek (but remember it forces full evaluation).


In [ ]:

g = (x for x in range(5))
print(list(g))
print(list(g))  # empty now — already consumed



## 9. Exercises (with hidden solutions)

Each task has a code cell for your answer. Open the **Show solution** block to compare.



### Exercise 1 — Simple range generator (Easy)

Write a generator function `my_range(start, stop, step)` that mimics a tiny subset of `range()`.
Yield numbers `start, start+step, ...` strictly less than `stop`.


In [ ]:

# Your turn!
def my_range(start, stop=None, step=1):
    # If called as my_range(stop), shift args like range(stop)
    if stop is None:
        start, stop = 0, start
    # TODO: yield values from start to stop (exclusive) stepping by step
    pass

# Quick checks:
print(list(my_range(5)))      # -> [0,1,2,3,4]
print(list(my_range(2, 7)))   # -> [2,3,4,5,6]
print(list(my_range(2, 10, 3)))  # -> [2,5,8]



<details>
<summary><strong>Show solution</strong></summary>

```python

def my_range(start, stop=None, step=1):
    if stop is None:
        start, stop = 0, start
    if step == 0:
        raise ValueError("step cannot be 0")
    i = start
    if step > 0:
        while i < stop:
            yield i
            i += step
    else:
        while i > stop:
            yield i
            i += step  # step is negative

```
</details>



### Exercise 2 — Even numbers up to N (Easy)

Create a **generator expression** that yields even numbers from `0` to `N` (inclusive). Store it in `evens`.


In [ ]:

N = 12
# Your generator expression here:
evens = None

print(list(evens))  # -> [0, 2, 4, 6, 8, 10, 12]



<details>
<summary><strong>Show solution</strong></summary>

```python

N = 12
evens = (x for x in range(N+1) if x % 2 == 0)
print(list(evens))

```
</details>



### Exercise 3 — Sliding window (Medium)

Write a generator `windows(iterable, size)` that yields consecutive tuples (windows) of length `size`.
Example: `windows([1,2,3,4], 3)` → `(1,2,3)`, `(2,3,4)`.


In [ ]:

from collections import deque

def windows(iterable, size):
    # TODO
    pass

print(list(windows([1,2,3,4], 3)))  # -> [(1,2,3), (2,3,4)]



<details>
<summary><strong>Show solution</strong></summary>

```python

from collections import deque

def windows(iterable, size):
    it = iter(iterable)
    dq = deque(maxlen=size)
    for x in it:
        dq.append(x)
        if len(dq) == size:
            yield tuple(dq)

```
</details>



### Exercise 4 — Prime numbers (Medium)

Implement `primes()` as an **infinite generator** that yields prime numbers in ascending order.
Then take the first 15 primes.


In [ ]:

def primes():
    # TODO: yield 2, 3, 5, 7, 11, ...
    pass

# Take first 15
out = []
for p in primes():
    out.append(p)
    if len(out) == 15:
        break

print(out)  # -> [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]



<details>
<summary><strong>Show solution</strong></summary>

```python

def primes():
    yield 2
    candidate = 3
    found = [2]
    import math
    while True:
        is_p = True
        r = int(math.isqrt(candidate))
        for q in found:
            if q > r:
                break
            if candidate % q == 0:
                is_p = False
                break
        if is_p:
            found.append(candidate)
            yield candidate
        candidate += 2  # skip even numbers

```
</details>



### Exercise 5 — File pipeline (Hard)

Using **only generators**, build a three‑stage pipeline to process lines of text:
1) `read_lines(path)` — yields stripped lines,
2) `only_numbers(lines)` — yields lines that contain only digits,
3) `as_ints(lines)` — converts them to integers.

Then compute the **sum of the first 100 integers** produced by the pipeline.


In [ ]:

# Your turn! You may reuse patterns from earlier sections.
def read_lines(path):
    pass

def only_numbers(lines):
    pass

def as_ints(lines):
    pass

# Demo input:
sample = ["001", "abc", "42", "  7 ", "x9", "100", "999"]
lines = (x for x in sample)  # pretend it's a file
# TODO: connect the pipeline and sum the first 100 integers



<details>
<summary><strong>Show solution</strong></summary>

```python

def read_lines(path_or_iter):
    # accept a path or any iterable of strings
    if isinstance(path_or_iter, str):
        with open(path_or_iter, "r", encoding="utf-8") as f:
            for line in f:
                yield line.strip()
    else:
        for line in path_or_iter:
            yield line.strip()

def only_numbers(lines):
    for line in lines:
        if line.isdigit():
            yield line

def as_ints(lines):
    for line in lines:
        yield int(line)

from itertools import islice

sample = ["001", "abc", "42", "  7 ", "x9", "100", "999"]
pipeline = as_ints(only_numbers(read_lines(sample)))
total = sum(islice(pipeline, 100))
print(total)  # -> 1142

```
</details>



### Exercise 6 — Coroutine‑style `send()` (Hard, optional)

Make a generator `collector()` that *receives* values via `.send(x)` and keeps a running total.
When `.close()` is called, it should print (or yield) the final total.


In [ ]:

def collector():
    # HINT: use a `while True:` loop and `val = yield total`
    pass

# Manual drive:
c = collector()
next(c)           # prime
c.send(10)
c.send(5)
c.send(2)
c.close()         # should show total=17



<details>
<summary><strong>Show solution</strong></summary>

```python

def collector():
    total = 0
    try:
        while True:
            val = yield total
            if val is not None:
                total += val
    except GeneratorExit:
        print("Final total:", total)

c = collector()
next(c)
c.send(10); c.send(5); c.send(2)
c.close()

```
</details>



## Wrap‑up

- Generators = lazy iterators built with `yield`.
- Generator expressions = compact lazy pipelines with parentheses.
- Use them for: streaming data, memory efficiency, readable pipelines.
- Explore `itertools` for battle‑tested building blocks.

Happy generating! ✨
